# 02. Data Merge & Spatialization

In [1]:
from pathlib import Path
import folium
import geopandas as gpd
import pandas as pd
from datetime import datetime 
import os
from glob import glob
import numpy as np

/root/venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_2169/1333799927.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  i

In [2]:
# papermill parameters cell

OUTPUT_WARNINGS = False
SAVE_MERGEDDATA = True

In [3]:
if OUTPUT_WARNINGS is False:
    import warnings

    warnings.filterwarnings("ignore")

### Data

In [4]:
# Data folders
INTERIM_FOLDER = 'data/interim/'
ADMBOUND_INTERIM_FOLDER = 'data/interim/AdmBound_interimdata/'
SPATIALIZED_FOLDER = "data/interim/Spatialization_interimdata/"
DEMOGRAPHIC_INTERIM_FOLDER = "data/interim/demographic_interimdata/"
MERGED_DEMOGRAPHIC_INTERIM_FOLDER = "data/interim/demographic_interimdata/merged_demographic_interimdata/"

# create folder data/raw/ if not exists
Path(SPATIALIZED_FOLDER).mkdir(parents=True, exist_ok=True)
Path(MERGED_DEMOGRAPHIC_INTERIM_FOLDER).mkdir(parents=True, exist_ok=True)

In [5]:
# Datasets 
ADMBOUND_INTERIMDATA = ADMBOUND_INTERIM_FOLDER + 'AdmBound_interimdata.gpkg'

# PART 1 - KPI INDEX DATA

## Data Merge

In [6]:
# read interim dataset of Administrative Boundaries into geodataframe
gdf = gpd.read_file(ADMBOUND_INTERIMDATA)

# Creat column CMUN out of CTOT
gdf['CMUN'] = gdf['CTOT'].str[2:]

In [7]:
# Create list with KPI names
KPI_list = [
    'D1',
    'D2',
    'D3',
    'D4',
    'ECR1',
    'ECR2',
    'ECR4',
    'ECR5',
    'M1',
    'M2',
    'M3',
    'M4',
    'W2',
    'W3',
    'POP21'
]

In [8]:
# DATA MANAGEMENT --> SCRIPT MOET WORDEN AFGEMAAKT, KLOPT NU NIET
dataframe_list = []

for file in glob(INTERIM_FOLDER + '*.csv'):
    # import interim datasets of CCIs
    file_name = os.path.basename(file)
    file_name = file_name.split('_', 1)[0]
    df = pd.read_csv(file)

    # DATA HOMOGENIZATION
    # Add extra digit to dataset['CMUN'] - if it contains less than 5 characters
    if 'CMUN' in df.columns:
        df["CMUN"] = df["CMUN"].apply(lambda x: '{0:0>5}'.format(x))
    
    # Add extra digit to dataset['CTOT'] - if it contains less than 7 characters
    elif 'CTOT' in df.columns :
                df['CTOT'] = df['CTOT'].apply(lambda x: '{0:0>7}'.format(x))
    else:
        pass

    globals()[f"CCI_{file_name}"] = df
    dataframe_list.append(globals()[f"CCI_{file_name}"])

In [9]:
# DATA MERGE
# Merge GeoDataFrame with seperate DataFrames per KPI
gdf_temporal1 = gpd.GeoDataFrame(CCI_D1.merge(gdf, how = 'right'))
gdf_temporal2 = gpd.GeoDataFrame(CCI_D2.merge(gdf_temporal1, how = 'right'))
gdf_temporal3 = gpd.GeoDataFrame(CCI_D3.merge(gdf_temporal2, how = 'right'))
gdf_temporal4 = gpd.GeoDataFrame(CCI_D4.merge(gdf_temporal3, how = 'right'))
gdf_temporal5 = gpd.GeoDataFrame(CCI_ECR1.merge(gdf_temporal4, how = 'right'))
gdf_temporal6 = gpd.GeoDataFrame(CCI_ECR2.merge(gdf_temporal5, how = 'right'))
gdf_temporal7 = gpd.GeoDataFrame(CCI_ECR4.merge(gdf_temporal6, how = 'right'))
gdf_temporal8 = gpd.GeoDataFrame(CCI_ECR5.merge(gdf_temporal7, how = 'right'))
gdf_temporal9 = gpd.GeoDataFrame(CCI_M1.merge(gdf_temporal8, how = 'right'))
gdf_temporal10 = gpd.GeoDataFrame(CCI_M2.merge(gdf_temporal9, how = 'right'))
gdf_temporal11 = gpd.GeoDataFrame(CCI_M3.merge(gdf_temporal10, how = 'right'))
gdf_temporal12 = gpd.GeoDataFrame(CCI_M4.merge(gdf_temporal11, how = 'right'))
gdf_temporal13 = gpd.GeoDataFrame(CCI_W2.merge(gdf_temporal12, how = 'right'))
gdf_temporal14 = gpd.GeoDataFrame(CCI_W3.merge(gdf_temporal13, how = 'right'))
gdf_temporal15 = gpd.GeoDataFrame(CCI_POP21.merge(gdf_temporal14, how = 'right'))
gdf_temporal15

,CMUN,POP21,CTOT,W3,W2,M4,M3,M2,M1,ECR5,ECR4,ECR2,ECR1,D4,D3,D2,D1,Municipality,geometry
0,03124,578.0,1003124,1.0,24.919786,0.173010,NaN,NaN,64863.137164,4.625000,11.857143,0.3,1.0,0.538462,0.940154,1.0,1.0,Sella,"MULTIPOLYGON (((-0.33310 38.62221, -0.33159 38..."
1,03125,605.0,1003125,0.0,47.108871,0.000000,NaN,NaN,140.277264,7.500000,14.666667,0.6,1.0,0.538462,0.857152,1.0,1.0,Senija,"MULTIPOLYGON (((0.02977 38.73416, 0.03122 38.7..."
2,46224,177.0,1046224,0.0,0.015979,0.000000,NaN,NaN,156.289531,13.000000,12.142857,0.0,0.0,0.307692,0.975460,1.0,1.0,Segart,"MULTIPOLYGON (((-0.39904 39.67404, -0.39887 39..."
3,08222,2459.0,0908222,1.0,97.650514,0.081334,NaN,NaN,164.738271,18.000000,17.428571,0.6,1.0,0.307692,0.965373,0.0,1.0,Sant Llorenç D'Hortons,"MULTIPOLYGON (((1.79248 41.47367, 1.79275 41.4..."
4,08223,2514.0,0908223,1.0,92.895522,0.039777,NaN,NaN,NaN,14.818182,13.268293,0.0,0.0,0.307692,0.967754,0.0,1.0,Sant Llorenç Savall,"MULTIPOLYGON (((2.01043 41.65840, 2.01065 41.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8300,15072,11004.0,1215072,0.0,NaN,0.027263,NaN,NaN,424.143438,11.000000,11.293103,0.6,1.0,0.000000,0.912822,1.0,1.0,Rianxo,"MULTIPOLYGON (((-8.80421 42.63694, -8.80423 42..."
8301,15073,26839.0,1215073,0.0,NaN,0.011178,4.669871,NaN,931.672662,6.529412,12.787879,0.0,0.0,0.615385,0.871187,1.0,1.0,Ribeira,"MULTIPOLYGON (((-9.05316 42.50715, -9.05311 42..."
8302,15074,4493.0,1215074,0.0,NaN,0.422880,NaN,NaN,253.839057,9.380952,9.569892,0.6,1.0,0.461538,0.901845,1.0,1.0,Rois,"MULTIPOLYGON (((-8.77455 42.82889, -8.77411 42..."
8303,15075,16382.0,1215075,0.0,NaN,0.140398,10.639681,NaN,622.400145,14.000000,14.642857,0.0,0.0,0.461538,0.926605,1.0,1.0,Sada,"MULTIPOLYGON (((-8.29739 43.38668, -8.29728 43..."


In [10]:
# filter columns that refer to KPIs
gdf_master = gdf_temporal15[[
    'CTOT', 
    'CMUN', 
    'Municipality',
    'geometry', 
    'D1',
    'D2',
    'D3',
    'D4', 
    'ECR1', 
    'ECR2',
    'ECR4',
    'ECR5', 
    'M1',
    'M2',
    'M3',
    'M4',
    'W2',
    'W3',
    'POP21']]

# Drop duplicate values (contain same values)
gdf_master = gdf_master.drop_duplicates(subset=['CTOT'])

In [11]:
gdf_master.describe()

if SAVE_MERGEDDATA is True:
    gdf_master.describe().to_csv("data/processed/CCI/Spain/03_index/descriptive_unprocessed_CCI_KPIs.csv", index=True)

In [12]:
gdf_master.describe()

,D1,D2,D3,D4,ECR1,ECR2,ECR4,ECR5,M1,M2,M3,M4,W2,W3,POP21
count,8217.000000,8217.000000,8131.000000,8217.000000,8217.000000,8217.000000,8217.000000,8217.000000,5.757000e+03,667.000000,1699.000000,8131.000000,8131.000000,8217.000000,8.131000e+03
mean,0.987587,0.707923,0.842776,0.468363,0.339662,0.161239,12.068130,6.884639,1.106894e+04,0.970890,29.724077,0.164850,507.891976,0.154436,5.827710e+03
std,0.110728,0.454745,0.302697,0.155268,0.473623,0.246846,4.133767,5.072302,6.473407e+05,2.851155,52.405494,0.640442,4504.115006,0.361388,4.784572e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.363422e+00,0.003105,0.002930,0.000000,0.000000,0.000000,3.000000e+00
25%,1.000000,0.000000,0.895671,0.384615,0.000000,0.000000,9.500000,4.000000,2.771100e+02,0.071674,3.228944,0.000000,2.578322,0.000000,1.530000e+02
50%,1.000000,1.000000,0.983444,0.538462,0.000000,0.000000,11.000000,5.954545,6.090777e+02,0.198814,11.289505,0.000000,16.373684,0.000000,5.230000e+02
75%,1.000000,1.000000,1.000000,0.538462,1.000000,0.300000,13.500000,8.000000,1.329192e+03,0.618059,31.723206,0.122186,106.713790,0.000000,2.416000e+03
max,1.000000,1.000000,1.000000,0.846154,1.000000,1.000000,51.000000,86.000000,4.895940e+07,37.037037,615.829312,40.000000,289605.714286,1.000000,3.305408e+06


## Export files

In [13]:
# exports the geodataframe into GeoPackage file
file_name = 'CCI_spatialization_interimdata'
data_format = '.gpkg'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    gdf_master.to_file(SPATIALIZED_FOLDER + export_name, driver='GPKG') 

In [14]:
# exports the geodataframe into Shapefile
file_name = 'CCI_spatialization_interimdata'
data_format = '.shp'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    gdf_master.to_file(SPATIALIZED_FOLDER + export_name) 

In [15]:
# export the geodataframe into csv file
file_name = 'CCI_spatialization_interimdata'
data_format = '.csv'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    gdf_master.to_csv(SPATIALIZED_FOLDER + export_name, index=False)

# PART 2 - DEMOGRAPHIC DATA

In [16]:
df_base = pd.DataFrame(gdf)

In [17]:
# DATA MANAGEMENT --> SCRIPT MOET WORDEN AFGEMAAKT, KLOPT NU NIET
dataframe_list = []

for file in glob(DEMOGRAPHIC_INTERIM_FOLDER + '*.csv'):
    # import interim datasets of CCIs
    df = pd.read_csv(file)

    # DATA HOMOGENIZATION
    # Add extra digit to dataset['CMUN'] - if it contains less than 5 characters
    if 'CMUN' in df.columns:
        df["CMUN"] = df["CMUN"].apply(lambda x: '{0:0>5}'.format(x))
    
    # DATA MERGE
    df_base = pd.merge(df_base, df, on='CMUN')

In [18]:
# filter columns
df_demographic_total = df_base[[
    "CTOT", 
    'POPULATION_2020',
    "POPULATION_DENSITY_KM2_2020",
    "POPULATION_PERC_NATURAL_GROWTH_2020",
    #"GENDER_PERC_POP_MALE_2020",
    "GENDER_PERC_POP_FEMALE_2020",
    "AGE_AVERAGE_2020",
    "AGE_PERC_POP_BELOW_18_2020",
    "AGE_PERC_POP_ABOVE_65_2020",   
    #"NATIONALITY_PERC_SPANISH_2020",
    "NATIONALITY_PERC_NONSPANISH_2020",
    "HOUSING_AVERAGE_HOUSEHOLD_SIZE_2020",
    "HOUSING_PERC_SINGLEPERSON_HOUSEHOLD_2020",
    'HOUSING_RESIDENT_BUILDINGS_PER_CAPITA_2011',
    "INCOME_PER_CAPITA_2020",
    "INCOME_PER_HOUSEHOLD_2020",
    'INCOME_PERC_UNEMPLOYMENT_BENEFITS_OF_AVERAGE_SALARY_2020',
    "WEALTH_GINI_2020",
    "DEBT_MUNICIPALITY_PER_CAPITA_2021",
    "ECONOMY_COMPANIES_PER_CAPITA_2020",
    'AGRI_LIVESTOCKUNITS_DENSITY_KM2_2020',
    'AGRI_CATTLEFARMS_DENSITY_KM2_2020',    
    'TOURISM_HOUSES_PER_CAPITA_2022',
    "geometry"
]]

In [19]:
# Drop duplicate values (contain same values)
df_demographic_total = df_demographic_total.drop_duplicates(subset=['CTOT'])
df_demographic_total

,CTOT,POPULATION_2020,POPULATION_DENSITY_KM2_2020,POPULATION_PERC_NATURAL_GROWTH_2020,GENDER_PERC_POP_FEMALE_2020,AGE_AVERAGE_2020,AGE_PERC_POP_BELOW_18_2020,AGE_PERC_POP_ABOVE_65_2020,NATIONALITY_PERC_NONSPANISH_2020,HOUSING_AVERAGE_HOUSEHOLD_SIZE_2020,...,INCOME_PER_CAPITA_2020,INCOME_PER_HOUSEHOLD_2020,INCOME_PERC_UNEMPLOYMENT_BENEFITS_OF_AVERAGE_SALARY_2020,WEALTH_GINI_2020,DEBT_MUNICIPALITY_PER_CAPITA_2021,ECONOMY_COMPANIES_PER_CAPITA_2020,AGRI_LIVESTOCKUNITS_DENSITY_KM2_2020,AGRI_CATTLEFARMS_DENSITY_KM2_2020,TOURISM_HOUSES_PER_CAPITA_2022,geometry
0,1003124,580,14.963296,-1.379310,47.495682,49.5,10.9,27.9,9.9,2.16,...,11331,25227,8.310466,26.7,0.000000,0.004228,0.000000,0.000000,0.020690,"MULTIPOLYGON (((-0.33310 38.62221, -0.33159 38..."
1,1003125,596,124.750660,0.167785,48.514851,45.4,16.6,22.0,31.2,2.46,...,9237,24133,15.829694,32.0,0.000000,0.091379,0.000000,0.000000,0.010067,"MULTIPOLYGON (((0.02977 38.73416, 0.03122 38.7..."
2,1046224,178,26.853259,-0.561798,48.587571,50.6,4.5,24.7,2.8,2.05,...,14118,30167,7.232556,29.4,0.106068,0.001562,0.000000,0.000000,0.005618,"MULTIPOLYGON (((-0.39904 39.67404, -0.39887 39..."
3,0908222,2456,126.627461,0.040717,49.429503,41.5,19.5,15.8,6.1,2.66,...,14809,39398,5.681910,27.1,0.000244,0.008965,0.000000,0.000000,0.000407,"MULTIPOLYGON (((1.79248 41.47367, 1.79275 41.4..."
4,0908223,2494,60.611088,-1.684042,49.781312,44.0,17.5,20.9,5.2,2.50,...,13084,32707,7.397622,27.3,0.015255,0.076140,0.000000,0.000000,0.001604,"MULTIPOLYGON (((2.01043 41.65840, 2.01065 41.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8082,1215072,11008,187.434532,-0.708576,51.184103,47.4,13.8,26.3,0.9,2.78,...,10644,29938,8.574475,22.9,0.130598,0.065013,1.675469,0.153244,0.004906,"MULTIPOLYGON (((-8.80421 42.63694, -8.80423 42..."
8083,1215073,26721,393.189225,-0.333071,50.680336,44.6,15.9,21.2,4.4,2.69,...,10866,29715,6.682154,28.0,0.091284,0.136991,3.634510,0.485582,0.009842,"MULTIPOLYGON (((-9.05316 42.50715, -9.05311 42..."
8084,1215074,4485,48.261203,-0.691193,52.246441,48.9,13.3,28.7,0.9,2.94,...,10458,31619,8.343337,23.5,0.000000,0.009206,33.850203,1.215946,0.002676,"MULTIPOLYGON (((-8.77455 42.82889, -8.77411 42..."
8085,1215075,16345,588.219944,-0.477210,51.040399,46.4,15.3,23.9,5.7,2.40,...,12628,30926,7.526393,32.7,0.001368,0.276923,0.809725,0.143951,0.005323,"MULTIPOLYGON (((-8.29739 43.38668, -8.29728 43..."


In [20]:
# Replace dots with empty
col_list = ["INCOME_PER_CAPITA_2020", "INCOME_PER_HOUSEHOLD_2020", "WEALTH_GINI_2020"]

for col in col_list:
    print(col)
    df_demographic_total[col] = df_demographic_total[col].astype('str')
    df_demographic_total.loc[df_demographic_total[col].str.startswith('.', na=False), col] = ' '
    df_demographic_total.loc[df_demographic_total[col].str.startswith(' ', na=False), col] = np.nan

    # Fill empty cells with NAN
    df_demographic_total[col] = df_demographic_total[col].astype('float')

INCOME_PER_CAPITA_2020
INCOME_PER_HOUSEHOLD_2020
WEALTH_GINI_2020


In [21]:
gdf_demographic_total = gpd.GeoDataFrame(df_demographic_total)

## Export files

In [22]:
# exports the geodataframe into GeoPackage file
file_name = 'Spatial_demographic_interimdata'
data_format = '.gpkg'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    gdf_demographic_total.to_file(MERGED_DEMOGRAPHIC_INTERIM_FOLDER + export_name, driver='GPKG') 

In [23]:
# exports the geodataframe into Shapefile
file_name = 'Spatial_demographic_interimdata'
data_format = '.shp'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    gdf_demographic_total.to_file(MERGED_DEMOGRAPHIC_INTERIM_FOLDER + export_name) 

In [24]:
# export the geodataframe into csv file
file_name = 'Spatial_demographic_interimdata'
data_format = '.csv'

export_name = file_name + data_format

if SAVE_MERGEDDATA is True:
    df_demographic_total.to_csv(MERGED_DEMOGRAPHIC_INTERIM_FOLDER + export_name, index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f11c3f49-2b5e-4e69-9186-d202c2d21fb7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>